In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from warcio.capture_http import capture_http
import requests  # requests must be imported after capture_http
from warcio.archiveiterator import ArchiveIterator
from warcio import WARCWriter
import io
import json

In [8]:
r = requests.get("https://www.eyedrum.org/calendar-events-performances-art-music/simone-baron-arco-belo-at-eyedrum")
f = open('test.html', 'w+')
f.write(r.text)
f.close()

In [4]:
with capture_http('test.warc.gz'):
    r = requests.get("https://fingerprinter-production.herokuapp.com/visitors/9da0eb5b2cac163039f1a7900f452aae/audience_segments")


### Read warc

In [11]:
warc = '../fidelity-files/collections/test/archive/patientsafety.va.gov_ed652f3b55.warc'
with open(warc, 'rb') as stream:
    for record in ArchiveIterator(stream):
        print(record.rec_type)
        print(record.rec_headers)
        print('\n', record.http_headers)
        print("\n\n")

warcinfo
WARC/1.1
WARC-Record-ID: <urn:uuid:c50856f0-42f9-5a22-a3f1-b6f2e8e9a47f>
WARC-Filename: test.warc
WARC-Date: 2024-05-08T19:53:53.280Z
WARC-Type: warcinfo
Content-Type: application/warc-fields
Content-Length: 115


 None



response
WARC/1.1
WARC-Record-ID: <urn:uuid:33a29a94-fddf-5558-aee4-52dde4378b13>
WARC-Page-ID: zomi2dnlzvrwhopfg9nnk9
WARC-Target-URI: https://cdn.eo.va.gov/va/2014/css/extended-styles.min.css
WARC-Date: 2024-05-14T22:04:06.272Z
WARC-Type: response
Content-Type: application/http; msgtype=response
WARC-Payload-Digest: sha256:afc94920b0d7581f294e2675583d852faae7e640b44619d3bd1a4b9bafa0dba0
WARC-Block-Digest: sha256:2d128bef23a80e275583986ac2e648153e12e24815cfa6bff743ac4ddd8238d4
Content-Length: 24419


 HTTP/1.1 200 OK
Accept-Ranges: bytes
Access-Control-Allow-Origin: *
Content-Length: 23891
Content-Type: text/css
Date: Tue, 14 May 2024 22:04:05 GMT
ETag: "80e15bea09bd31:0"
Last-Modified: Thu, 01 Feb 2018 21:08:01 GMT
Server: 
Vary: Accept-Encoding
X-Frame-Op

### Query pywb's CDX

In [10]:
{
    'url': 'https://patientsafety.va.gov/va_files/2014/responsive/images/social-email.png',
    'closest': '20240514173219', 
    'filter': ['!mime:warc/revisit', 'digest:sha-256:20c06d50d7c70b6a1ac9a431064fc5110135d8830c9b6e57541374ac057d2ac3']
}
coll = 'test'
base_url = f'http://pistons.eecs.umich.edu:8080/{coll}/cdx'
params = {
    'url': 'https://patientsafety.va.gov/va_files/2014/responsive/images/social-email.png',
    'closest': '20240514173219', 
    'filter': ['!mime:warc/revisit', 'digest:sha-256:20c06d50d7c70b6a1ac9a431064fc5110135d8830c9b6e57541374ac057d2ac3'],
    'output': 'json'
}
r = requests.get(base_url, params=params)
print(json.dumps(r.json(), indent=2))

{
  "urlkey": "gov,va,patientsafety)/va_files/2014/responsive/images/social-email.png",
  "timestamp": "20240514173219",
  "url": "https://patientsafety.va.gov/va_files/2014/responsive/images/social-email.png",
  "mime": "image/png",
  "status": "200",
  "digest": "sha256:20c06d50d7c70b6a1ac9a431064fc5110135d8830c9b6e57541374ac057d2ac3",
  "length": "2188",
  "offset": "648854",
  "filename": "patientsafety.va.gov_ed652f3b55.warc",
  "source": "test/indexes/index.cdxj",
  "source-coll": "test",
  "is_fuzzy": "1",
  "access": "allow"
}


### Replace target warc

In [5]:
warc = '/vault-swift/jingyz/golfdigest.warc'
new_warc = '/vault-swift/jingyz/golfdigest-withissue.warc'
target_url_q = 'golfdigest-frontend'
replace = lambda x: x.replace('o=e.getConfig()', 'o=null')

with open(warc, 'rb') as stream, open(new_warc, 'wb') as out:
    writer = WARCWriter(out, gzip=False)
    for record in ArchiveIterator(stream):
        uri = record.rec_headers.get_header('WARC-Target-URI')
        body = record.content_stream().read()
        if uri and target_url_q in uri and record.rec_type == 'response':
            print(body)
            old_body = body
            body = replace(body.decode())
            body = body.encode()
            print(old_body == body)
        new_record = writer.create_warc_record(record.rec_headers.get_header('WARC-Target-URI'), 
                                            record.rec_type,
                                            payload=io.BytesIO(body),
                                            warc_headers=record.rec_headers,
                                            http_headers=record.http_headers)
        writer.write_record(new_record)

b'!function(e){var t={};function n(r){if(t[r])return t[r].exports;var i=t[r]={i:r,l:!1,exports:{}};return e[r].call(i.exports,i,i.exports,n),i.l=!0,i.exports}n.m=e,n.c=t,n.d=function(e,t,r){n.o(e,t)||Object.defineProperty(e,t,{enumerable:!0,get:r})},n.r=function(e){"undefined"!=typeof Symbol&&Symbol.toStringTag&&Object.defineProperty(e,Symbol.toStringTag,{value:"Module"}),Object.defineProperty(e,"__esModule",{value:!0})},n.t=function(e,t){if(1&t&&(e=n(e)),8&t)return e;if(4&t&&"object"==typeof e&&e&&e.__esModule)return e;var r=Object.create(null);if(n.r(r),Object.defineProperty(r,"default",{enumerable:!0,value:e}),2&t&&"string"!=typeof e)for(var i in e)n.d(r,i,function(t){return e[t]}.bind(null,i));return r},n.n=function(e){var t=e&&e.__esModule?function(){return e.default}:function(){return e};return n.d(t,"a",t),t},n.o=function(e,t){return Object.prototype.hasOwnProperty.call(e,t)},n.p="",n(n.s=17)}([,function(e,t,n){\n/*! @license DOMPurify | (c) Cure53 and other contributors | Relea